## VPRTempo & VPRTempoQuant - Basic Demo

### By Adam D Hines (https://research.qut.edu.au/qcr/people/adam-hines/)

VPRTempo is based on the following paper, if you use or find this code helpful for your research please consider citing the source:
    
[Adam D Hines, Peter G Stratton, Michael Milford, & Tobias Fischer. "VPRTempo: A Fast Temporally Encoded Spiking Neural Network for Visual Place Recognition. arXiv September 2023](https://arxiv.org/abs/2309.10225)

### Introduction

This is a basic, extremely simplified version of VPRTempo that highlights how images are transformed, spikes and weights are used, and the readout for performance using a model trained using our base system and the Quantized Aware Training (QAT) version. This is a basic, extremely simplified version of VPRTempo that highlights how images are transformed, spikes and weights are used, and the readout for performance. Although the proper implementation is in [PyTorch](https://pytorch.org/), we present a simple NumPy example to get started. As in the paper, we will present a simple example using the [Nordland](https://webdiis.unizar.es/~jmfacil/pr-nordland/#download-dataset) dataset with pre-trained set of weights.

Before starting, make sure the following packages are installed and imported:

In [ ]:
# Imprt opencv-python, NumPy, and matplotlib.pyplot
try:
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
except:
    !pip3 install numpy, opencv-python, matplotlib # pip install if modules not present
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt

Next, we will need to get the pretrained weights for the model. To get them and the other materials for the , we will use [Git Large File Storage](https://git-lfs.com/) to download the required materials. Please ensure you have it downloaded on your local machine prior to running these tutorials.

In [ ]:
import subprocess

# Run Git LFS pull command
try:
    result = subprocess.run("git lfs pull", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    print("Git LFS pull successful.")
except subprocess.CalledProcessError as e:
    print(f"Git LFS pull failed with error:\n{e.stderr}")

### Image processing

Let's have a look at how we process our images to run through VPRTempo. We utilize a technique called *patch normalization* to resize input images and normalize the pixel intensities. To start, let's see what the original image looks like before patch normalization.

In [ ]:
# Load the input image
raw_img = cv2.imread('./mats/1_BasicDemo/summer.png')
rgb_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB) # Convert to RGB

# Plot the image
plt.imshow(rgb_img)
plt.title('Nordland Summer')
plt.show()

What we have here is a 360x640 RGB image, which for processing through neural networks is too big (230,400 total pixels). So instead, we'll use patch normalization to reduce the image size down to a grayscale 56x56 image to just 3136 pixels in total.

In [ ]:
# Load the patch normalized image
patch_img = np.load('./mats/1_BasicDemo/summer_patchnorm.npy', allow_pickle=True)

# Plot the image
plt.matshow(patch_img)
plt.title('Nordland Summer Patch Normalized')
plt.colorbar(shrink=0.75, label="Pixel intensity")
plt.show()

The reduced image dimensions with patch normalization allows for a decent representation of the full scene, despite the smaller size.

### Convert images to spikes

'Spikes' in the context of VPRTempo are a little different than conventional spiking neural networks. Typically, spikes from image datasets are converted into Poisson spike trains where the pixel intensity determines the number of spikes to propagate throughout a network. VPRTempo only considers each pixel as a single spike, but considers the *amplitude* of the spike to determine the timing within a single timestep - where large amplitudes (high pixel intensity) spike early in a timestep, and vice versa for small amplitudes. 

Let's flatten the patch normalized image into a 1D-array so we can apply our network weights.

In [ ]:
# Convert 2D image to a 1D-array
patch_1d = np.reshape(patch_img, (3136,))

### Load the pre-trained network weights

Our network consists of the following architecture:

    - An input layer sparsely connected to a feature layer, 3136 input neurons to 6272 feature neurons
    - The feature layer fully connected to a one-hot-encoded output layer, 6272 feature neurons to 500 output neurons

Each layer connection is trained separately and stored in different weight matrices for excitatory (positive) and inhibitory (negative) connections. 

In [ ]:
# Load the input to feature excitatory and inhibitory network weights
featureW = np.load('./mats/1_BasicDemo/featureW.npy')

# Plot the  weights
plt.matshow(featureW.T)
plt.title('Input > Feature Weights')
plt.colorbar(shrink=0.8, label="Weight strength")

# Display the plots
plt.show()

Whilst it might be a little difficult to see, our excitatory connection amplitudes are on average a little higher than our inhibitiory. However, we overall have more inhibitiory connections that positive to balance the system.

This is because when we set up our connections we use a probability of connections for both excitation and inbhition. In this case, we have a 10% connection probability for excitatory weights and a 50% probability for inhibitiory. This means as well there will be a high number of neurons without connections.

In [ ]:
# In this function, we will plot and visualize the distribution of weights and connections.
def count_and_plot(array):
    # Flatten the 2D array and count positive, negative, and zero values
    flattened_array = array.flatten()
    positive_count = np.sum(flattened_array > 0)
    negative_count = np.sum(flattened_array < 0)
    zero_count = np.sum(flattened_array == 0)
    
    # Calculate percentages
    total_count = flattened_array.size
    positive_percentage = (positive_count / total_count) * 100
    negative_percentage = (negative_count / total_count) * 100
    zero_percentage = (zero_count / total_count) * 100

    # Print the results
    print(f"Excitatory Connections: {positive_count} ({positive_percentage:.2f}%)")
    print(f"Inhibitory Conncetions: {negative_count} ({negative_percentage:.2f}%)")
    print(f"Zero Connections: {zero_count} ({zero_percentage:.2f}%)")

    # Create a bar plot of the percentages
    categories = ['Excitatory', 'Inhibitory', 'Zero']
    percentages = [positive_percentage, negative_percentage, zero_percentage]

    plt.bar(categories, percentages)
    plt.xlabel('Category')
    plt.ylabel('Percentage')
    plt.title('Percentage of Excitatory, Inhibitiory, and Zero Connections')
    plt.ylim(0, 60)  # Set the y-axis limit to 0-60%
    plt.show()

if __name__ == "__main__":

    # Call the function to count and plot
    count_and_plot(featureW)

Now let's have a look at the feature to the output weights, and see how the distribution of excitiatory and inhibitory connections differs.

In [ ]:
# Load the input to feature excitatory and inhibitory network weights
outputW = np.load('./mats/1_BasicDemo/outputW.npy')

# Plot the  weights
plt.matshow(outputW)
plt.title('Feature > Output Weights')
plt.colorbar(shrink=0.8, label="Weight strength")

# Display the plots
plt.show()

# Plot the distributions
count_and_plot(outputW)

### Propagate network spikes

Now we'll propagate the input spikes across the layers to get the output. All we have to do is multiply the input spikes by the Input > Feature weights for both excitatory and inhibitory matrices and add them, then take the feature spikes and multiply them by the Feature > Output weights and do the smae thing. We'll also clamp spikes in the range of [0, 0.9] to prevent negative spikes and spike explosions.

Let's do that and visualize the spikes as they're going through, we'll start with the Input to Feature layer.

In [ ]:
# Calculate feature spikes (positive and negative weights)
feature_spikes = np.matmul(featureW,patch_1d)
feature_spikes = np.clip(feature_spikes, 0, 0.9)

# Now create the line plot
plt.plot(np.arange(len(feature_spikes)), feature_spikes)

# Add title and labels if you wish
plt.title('Feature Layer Spikes')
plt.xlabel('Neuron ID')
plt.ylabel('Spike Amplitude')

# Show the plot
plt.show()

This looks a little homogenous, but this is the feature representation of our input image. 

Now let's propagate the feature layer spikes through to the output layer to get our corresponding place match.

In [ ]:
# Calculate output spikes (positive and negative weights)
output_spikes = np.matmul(outputW,feature_spikes)
output_spikes = np.clip(output_spikes, 0, 0.9)

# Now create the line plot
plt.plot(np.arange(len(output_spikes)), output_spikes)

# Add title and labels if you wish
plt.title('Output Layer Spikes')
plt.xlabel('Neuron ID')
plt.ylabel('Spike Amplitude')

# Show the plot
plt.show()

Success! We have propagated our input spikes across the layers to reach this output. Clearly, one of the output spikes has the highest amplitude. Our network weights were trained on 500 locations from a Fall and Spring traversal of Nordland. For this example, we passed the first location from the Summer traversal through the network to achieve this output - which clearly looks to have spikes Neuron ID '0' the highest!

Let's prove that.

In [ ]:
# Output the argmax from the output spikes
prediction = np.argmax(output_spikes)
print(f"Neuron ID with the highest output is {prediction}")

## Quantized model example

Now that we have seen how our base model works, let's look at how our int8 quantized model performs by comparison. Working in the in8 space has a few benefits, like faster inferencing time and smaller model sizes. There are a couple differences however when feeding spikes throughout the system that PyTorch performs in the backend.

Let's start by converting our input image into int8 spikes.

In [ ]:
# Converting fp32 spikes to int8 uses a learned scale factor during quantization aware training
spike_scale = 133
patch_img_int = patch_img*spike_scale

# Plot the converted int8 image
plt.matshow(patch_img_int)
plt.title('Nordland Summer Patch Normalized Int8')
plt.colorbar(shrink=0.75, label="Pixel intensity")
plt.show()

# Convert 2D image to a 1D-array
patch_1d_int = np.reshape(patch_img_int, (3136,))

Now we'll load in and plot our integer based weights, as well as some scale factors which will be important to reduce the size of our spikes after multiplying them with our weights.

In [ ]:
# Load the scales for the feature and output spikes
feature_scales = np.load('./mats/1_BasicDemo/featureScales.npy',allow_pickle=True)
output_scales = np.load('./mats/1_BasicDemo/outputScales.npy',allow_pickle=True)

# Load the int8 weights and plot them
featureQuantW = np.load('./mats/1_BasicDemo/featureQuantW.npy')
outputQuantW = np.load('./mats/1_BasicDemo/outputQuantW.npy')

# Plot the feature weights
plt.matshow(featureQuantW.T)
plt.title('Input > Feature Weights')
plt.colorbar(shrink=0.8, label="Weight strength")

# Display the plots
plt.show()

# Plot the output weights
plt.matshow(outputQuantW)
plt.title('Feature > Output Weights')
plt.colorbar(shrink=0.8, label="Weight strength")

# Display the plots
plt.show()

Now as above, let's propagate the input spikes throughout the network.

In [ ]:
# Get the feature spikes
feature_spikes_int = np.matmul(featureQuantW,patch_1d_int)

# Now create the line plot
plt.plot(np.arange(len(feature_spikes_int)), feature_spikes_int)

# Add title and labels if you wish
plt.title('Output Layer Spikes')
plt.xlabel('Neuron ID')
plt.ylabel('Spike Amplitude')

# Show the plot
plt.show()

Those are some big spikes! We're going to have to scale these spikes back down before we forward them to the output layer, otherwise we'll have some huge activations. Let's take those scales we loaded in earlier and apply them to the feature spikes.

We have three things to consider here:
 - A slice scale factor (per neuronal connection scale)
 - A zero point (a factor to change where 'zero' is)
 
 Let's print out these three factors and see how they scale our spikes.

In [ ]:
# Print out the individual scales
print(f"The slice scale factor is {feature_scales[1]}")
print(f"The zero point is {feature_scales[2]}")

Now we'll modify and scale our spikes to then pass them on to the feature layer.

In [ ]:
# Scale the feature spikes
scaled_feature_spikes = (feature_spikes_int//(feature_scales[1]))+feature_scales[2]
scaled_feature_spikes = np.clip(scaled_feature_spikes,0,255)

# Plot the scaled feature spikes
plt.plot(np.arange(len(scaled_feature_spikes)), scaled_feature_spikes)

# Add title and labels if you wish
plt.title('Output Layer Spikes')
plt.xlabel('Neuron ID')
plt.ylabel('Spike Amplitude')

# Show the plot
plt.show()

Now that we've scaled our feature spikes, let's pass them through to the output layer and get our match!

In [ ]:
# Get the output spikes
output_spikes_int = np.matmul(outputQuantW,scaled_feature_spikes)

# Scale the output spikes
scaled_output_spikes = output_spikes_int//(output_scales[1]) + output_scales[2]

# Plot the scaled feature spikes
plt.plot(np.arange(len(scaled_output_spikes)), scaled_output_spikes)

# Add title and labels if you wish
plt.title('Output Layer Spikes')
plt.xlabel('Neuron ID')
plt.ylabel('Spike Amplitude')

# Show the plot
plt.show()

And once again, as in the base model, we can see that output neuron 0 is the highest respondant.

Let's prove it!

In [ ]:
# Output the argmax from the output spikes
prediction = np.argmax(scaled_output_spikes)
print(f"Neuron ID with the highest output is {prediction}")

### Conclusions

We have gone through a very basic demo of how VPRTempo takes input images, patch normalizes them, and propagates the spikes throughout the weights to achieve the desired matching output. Although this demonstration was performed using NumPy, the torch implementation is virtually the same except we use tensors with or without quantization. 

We also went through how the quantization version of the network handled weights and spikes in the integer domain.

If you would like to go more in-depth with training and inferencing, checkout some of the [other tutorials](https://github.com/AdamDHines/VPRTempo-quant/tree/main/tutorials) which show you how to train your own model and goes through the more sophisticated implementation of VPRTempo.